In [1]:
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [2]:
%load_ext autoreload
%autoreload 2
from analysis_toolbox import *

In [92]:
import ast
from qualityBaseline import * 
from pycocotools.coco import COCO
import matplotlib.image as mpimg
save_db_as_csv(connect=False)
img_info,object_tbl,bb_info,hit_info = load_info()

In [4]:
all_x_locs = bb_info["x_locs"]
all_y_locs = bb_info["y_locs"]

In [7]:
dataDir='../../coco/'
dataType='train2014'
annFile='%s/annotations/instances_%s.json'%(dataDir,dataType)
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=15.76s)
creating index...
index created!


In [9]:
ground_truth = pd.read_csv("../../data/object_ground_truth.csv")

In [10]:
def process_raw_locs(segmentation,COCO=False):
    '''
    Given a raw string of x and y coordinates, process it
    return a list of x_locs and y_locs
    '''
    x_locs=[]
    y_locs=[]
    if COCO:
        #COCO
#         print "Process COCO"
        poly = np.array(segmentation).reshape((len(segmentation)/2, 2))
        x_locs = list(poly[:,0])
        y_locs = list(poly[:,1])
    else: 
        bbx_path,bby_path = segmentation
        x_locs = [x for x in ast.literal_eval(bbx_path) if x is not None]
        y_locs = [y for y in ast.literal_eval(bby_path) if y is not None]


    # Append the starting point again in the end to close the BB
    x_locs.append(x_locs[0])
    y_locs.append(y_locs[0])
    return x_locs,y_locs

In [11]:
bb_info['COCO Majority Vote'] = pd.Series(np.zeros(len(bb_info)), index=bb_info.index)

In [32]:
bb_info.ix[bb[0]]

object_id                                                             8
worker_id                                                             3
x_locs                [302, 264, 254, 250, 250, 250, 248, 247, 247, ...
y_locs                [124, 131, 137, 145, 152, 160, 167, 173, 188, ...
COCO Majority Vote                                                    0
Name: 0, dtype: object

In [93]:
for i in np.arange(len(img_info)):
    img_name = img_info["filename"][i]
    if 'COCO' in img_name:
        fname = "../web-app/app/static/"+img_name+".png"
        img=mpimg.imread(fname)
        width,height = get_size(fname)
        img_id = int(img_name.split('_')[-1])
        filtered_object_tbl = object_tbl[object_tbl["image_id"]==i+1]
        annIds = coco.getAnnIds(imgIds=img_id, iscrowd=None)
        anns = coco.loadAnns(annIds)
        #for oid,bbx_path,bby_path in zip(bb_info["object_id"],bb_info["x_locs"],bb_info["y_locs"]):
        for bb in bb_info.iterrows():
            oid = bb[1]["object_id"]
            bbx_path= bb[1]["x_locs"]
            bby_path= bb[1]["y_locs"]
            if int(object_tbl[object_tbl.object_id==oid].image_id) ==i+1:
                worker_x_locs,worker_y_locs= process_raw_locs([bbx_path,bby_path])
                ground_truth_match = ground_truth[ground_truth.id==str(oid)]
                COCO_id = int(ground_truth_match["COCO_annIds"])
                
                #COCO-Annotations
                for ann in anns:
                    if COCO_id==-1:
                        #No BB for this object collected by MSCOCO
                        pass
                    elif ann['id'] == COCO_id: 
#                         print COCO_id
                        for annBB in ann["segmentation"]:
                            bbg_x_locs,bbg_y_locs = process_raw_locs(annBB,COCO=True)
    #                         print ann["segmentation"]
#                             print majority_vote([worker_x_locs,bbg_x_locs],[worker_y_locs,bbg_y_locs],width,height)                
                            obj_x_locs = [worker_x_locs,bbg_x_locs]
                            obj_y_locs = [worker_y_locs,bbg_y_locs]
                            bb_info = bb_info.set_value(bb[0],"COCO Majority Vote",majority_vote(obj_x_locs,obj_y_locs,width,height))                
                            #bb_info = bb_info.set_value(bb[0],"Munkres Euclidean",DistAllWorkers(obj_x_locs,obj_y_locs))
                            bb_info = bb_info.set_value(bb[0],"COCO Precision",precision(obj_x_locs,obj_y_locs,width,height))                
                            bb_info = bb_info.set_value(bb[0],"COCO Recall",recall(obj_x_locs,obj_y_locs,width,height))                

- Work on debugging Munkres Euclidean later 

### Precision and recall 

In [59]:
def make_iou_mask(obj_x_locs,obj_y_locs,width,height):
    '''
    Returns a mask that has all the worker's responses overlaid on top of each other
    '''
    iou_mask = np.zeros((height,width))
    img = Image.new('L', (width, height), 0)
    for x_locs, y_locs in zip(obj_x_locs,obj_y_locs):
        ImageDraw.Draw(img).polygon(zip(x_locs,y_locs), outline=1, fill=1)
        mask = np.array(img)==1
        iou_mask+=mask
    return iou_mask

In [60]:
from PIL import Image, ImageDraw
def majority_vote(obj_x_locs,obj_y_locs,width,height): 
    '''
    Jaccard Simmilarity or Overlap Method
    used for PASCAL VOC challenge
    '''
    iou_mask  = make_iou_mask(obj_x_locs,obj_y_locs,width,height) 
    # Compute Jaccard Simmilarity 
    intersection = len(np.where(iou_mask == iou_mask.max())[0])
    union  =len(np.where(iou_mask !=0)[0])
    return float(union)/intersection

In [62]:
import qualityBaseline

In [64]:
majority_vote(obj_x_locs,obj_y_locs,width,height) == qualityBaseline.majority_vote(obj_x_locs,obj_y_locs,width,height)

True

In [85]:
def area(obj_x_loc,obj_y_loc,width,height,debug = False):
    iou_mask = np.zeros((height,width))
    img = Image.new('L', (width, height), 0)
    ImageDraw.Draw(img).polygon(zip(obj_x_loc,obj_y_loc), outline=1, fill=1)
    iou_mask = np.array(img)==1
    if debug: 
        plt.imshow(iou_mask)
        plt.colorbar()
    area = len(np.where(iou_mask==1)[0])
    return area

In [86]:
def recall(obj_x_locs,obj_y_locs,width,height):
    iou_mask  = make_iou_mask(obj_x_locs,obj_y_locs,width,height) 
    intersection = len(np.where(iou_mask == iou_mask.max())[0])
    worker_bb_area  = area(obj_x_locs[0],obj_y_locs[0],width,height)
    return intersection/float(worker_bb_area)

In [87]:
def precision(obj_x_locs,obj_y_locs,width,height):
    iou_mask  = make_iou_mask(obj_x_locs,obj_y_locs,width,height) 
    intersection = len(np.where(iou_mask == iou_mask.max())[0])
    truth_bb_area  = area(obj_x_locs[1],obj_y_locs[1],width,height)
    return intersection/float(truth_bb_area)

In [88]:
precision(obj_x_locs,obj_y_locs,width,height)

2.084280303030303

In [89]:
recall(obj_x_locs,obj_y_locs,width,height)

1.0

if total recall, that means the BB is overbounding, or maybe our ground truth is just not very good

In [98]:
# replace all NAN values with -1, these are entries for which we don't have COCO ground truth
bb_info = bb_info.fillna(-1)